In [ ]:
Take a hamiltonian, and encode its groundstate into a quantum circuit leveraging tensor network methods.


# Load Julia’s package manager interface
using Pkg

# Install Jupyter notebook integration for Julia
Pkg.add("IJulia")

# Install the core ITensors library for tensor network calculations
Pkg.add("ITensors")

# Install the ITensorMPS extension for working specifically with Matrix Product States (MPS)
Pkg.add("ITensorMPS")

# Install the standard library module for linear algebra routines
Pkg.add("LinearAlgebra")

# Install Julia’s built-in random number utilities
Pkg.add("Random")

# Install formatted string printing (similar to printf in C)
Pkg.add("Printf")


# Load the modules into the current session:

# ITensors: core types and operations for tensor networks
using ITensors

# ITensorMPS: higher-level MPS-specific routines built on ITensors
using ITensorMPS

# LinearAlgebra: matrix factorizations, norms, inner products, eigenvalues, etc.
using LinearAlgebra





"""
    generate_mps_xyz(
        N::Int,
        Jx::Float64,
        Jy::Float64,
        Jz::Float64,
        h::Float64,
        nsweeps::Int,
        cutoff::Vector{Float64},
    ) → (MPS, Float64)

Constructs the spin-½ XYZ chain Hamiltonian with a transverse field, runs DMRG to find its ground state, 
and returns a left-canonical Matrix Product State (MPS) along with the ground-state energy.

# Arguments
- `N::Int`  
  Number of spins in the chain.
- `Jx, Jy, Jz::Float64`  
  Exchange couplings along the x, y, and z spin components.
- `h::Float64`  
  Strength of the transverse field in the x-direction.
- `nsweeps::Int`  
  Number of DMRG sweeps to perform.
- `cutoff::Vector{Float64}`  
  Truncation thresholds for each sweep (controls bond-dimension growth).

# Returns
- `psi_lcanon::MPS`  
  The ground-state MPS in left-canonical form.
- `energy::Float64`  
  The computed ground-state energy eigenvalue.
"""
function generate_groundstate_mps(
    N::Int,
    Jx::Float64,
    Jy::Float64,
    Jz::Float64,
    h::Float64,
    nsweeps::Int,
    cutoff::Vector{Float64},
)
    # 1) Define the spin-½ site indices for an N-site chain
    sites = siteinds("S=1/2", N)

    # 2) Build the Hamiltonian as an Operator Sum (OpSum) - the user has complete liberty in chosing the hamiltonian
    os = OpSum()
    #   — nearest-neighbor XYZ exchange terms
    for j in 1:N-1
        os += Jx, "Sx", j, "Sx", j+1   # Jx Sx_j Sx_{j+1}
        os += Jy, "Sy", j, "Sy", j+1   # Jy Sy_j Sy_{j+1}
        os += Jz, "Sz", j, "Sz", j+1   # Jz Sz_j Sz_{j+1}
    end
    #   — transverse field in x-direction on every site
    for j in 1:N
        os += -h, "Sx", j              # -h Sx_j
    end
    #   Convert the OpSum into an MPO over our site indices
    H = MPO(os, sites)

    # 3) Initialize a random product MPS (all spins “Dn” = spin-down)
    psi0 = random_mps(sites, n -> "Dn")

    # 4) Perform the DMRG algorithm
    #   Returns (energy, final MPS) after `nsweeps` with given `cutoff` thresholds
    energy, psi1 = dmrg(H, psi0; nsweeps=nsweeps, cutoff=cutoff)

    # 5) Put the MPS into left-canonical form for convenience/storage
    psi_lcanon = orthogonalize!(psi1, 1)

    return psi_lcanon, energy
end

# ————————————————————————————————————————————————
# Example usage:

N       = 10              # number of spins
Jx      = 5.0             # coupling in x
Jy      = 6.0             # coupling in y
Jz      = 5.0             # coupling in z
h       = 0.5             # transverse field strength
nsweeps = 10              # how many DMRG sweeps
cutoff  = [1e-10]         # truncation threshold per sweep

# Run the DMRG and obtain ground state MPS + energy
psi_non_truncated, Energy = generate_groundstate_mps(N, Jx, Jy, Jz, h, nsweeps, cutoff)

# Required in future computations
sites = siteinds(psi_non_truncated)
product_state = MPS(sites, "Up")




"""
    generate_mpds(psi::MPS) → Dict{Int, ITensor}

Convert a given Matrix Product State (MPS) into a single “layer” of two-site unitary gates and a single-site unitary gate. 
Keep in mind this works only for 2 state systems (d = 2)
Each gate captures the entanglement structure on one bond of the chain: 
- the first loop handles the left boundary bond,  
- the second loop handles all bulk bonds, and  
- the final loop handles the right boundary (trivial single-site) gate.

# Arguments
- `psi::MPS` ->  this is the ground state MPS psi_non_truncated
  Input MPS of length N.

# Returns
- `mpds::Dict{Int, ITensor}`  
  A mapping from bond index `n` to an ITensor representing the two-site (or boundary) gate on sites `(n, n+1)` (or the single-site at `n = N`).
"""
function generate_mpds(psi)
    # Number of sites in the MPS
    N = length(psi_non_truncated)
    # Work on a copy so original psi is not mutated
    psi = copy(psi_non_truncated)

    # Container for the “unitary” gates/MPO tensors
    mpds = Dict{Int, ITensor}()

    # ——————— Left boundary bond (sites 1) ———————
    for n = 1
        # physical index at site 1
        i = inds(psi[n], "Site")[1]
        # bond index between site 1 and 2
        j = commoninds(psi[n], psi[n+1])[1]

        # Extract the 2-index tensor for site 1
        Array_tensor1 = Array(psi[n], i, j)

        # Embed into a 4-index array of shape (2,2,2,2),
        # padding zeros so we can form a full-rank block
        T = zeros(2, 2, 2, 2)
        T[:, :, 1, 1] = Array_tensor1

        # Build “output” (primed) and “input” (unprimed) indices for sites 1 & 2
        i2, j2 = prime(siteinds(psi, n)...), prime(siteinds(psi, n+1)...)
        k2, l2 = siteinds(psi, n), siteinds(psi, n+1)

        # Wrap into an ITensor with the 4 indices
        tensor = ITensor(T, i2, j2, k2, l2)
        # SVD across the cut between (i2,j2) and (k2,l2)
        U, S, V = svd(tensor, i2, j2)

        # Build delta to absorb S into the two halves
        S1 = delta(commonind(U, S), commonind(S, V))

        # Store the two-site unitary gate for bond 1
        mpds[n] = U * S1 * V
    end

    # ——————— Bulk bonds (sites 2 through N-1) ———————
    for n in 2:N-1
        # physical index at site n
        i = inds(psi[n], "Site")[1]
        # bond to the right (n ↔ n+1)
        j = commoninds(psi[n], psi[n+1])[1]
        # bond to the left (n-1 ↔ n)
        k = commoninds(psi[n-1], psi[n])[1]

        # Extract the 3-index tensor at site n
        Array_tensor2 = Array(psi[n], i, j, k)

        # Embed into a 4-index array (padding last leg)
        G = zeros(2, 2, 2, 2)
        G[:, :, :, 1] = Array_tensor2

        # Define primed/unprimed indices for sites n & n+1
        i2, j2 = prime(siteinds(psi, n)...), prime(siteinds(psi, n+1)...)
        k2, l2 = siteinds(psi, n), siteinds(psi, n+1)

        # Wrap and SVD as before
        tensor = ITensor(G, i2, j2, k2, l2)
        U, S, V = svd(tensor, i2, j2)
        S1 = delta(commonind(U, S), commonind(S, V))

        # Store the two-site gate for bond n
        mpds[n] = U * S1 * V
    end

    # ——————— Right boundary (site N single-site gate) ———————
    for n = N
        # physical index at site N
        i = inds(psi[n], "Site")[1]
        # bond to the left (N-1 ↔ N)
        j = commoninds(psi[n-1], psi[n])[1]

        # Replace the MPS tensor by a “gate” that simply relabels
        # its input/output indices (no entanglement at the boundary)
        mpds[n] = replaceinds(
            psi[n],
            i => prime(i),                    # output leg
            j => inds(psi[n], "Site")[1]      # bring bond back to a physical leg
        )
    end

    return mpds
end




"""
    dagger!(mpds::Vector{ITensor}) → Vector{ITensor}

Given a list of MPD gates `mpds`, this function computes their Hermitian conjugates,
swaps the “primed” index labels so they can be applied as inverse operations,
and reverses the list order. This produces a gate sequence `dag(U)` that
disentangles the MPS (psi_non_truncated) towards the product state |0>

# Arguments
- `mpds::Vector{ITensor}`  
  Sequence of two-site (or boundary) ITensor gates.

# Returns
- `mpds_dag::Vector{ITensor}`  
  The reversed, conjugated, and primed-index-swapped gate list.
"""
function dagger!(mpds)
    # 1) Take the Hermitian conjugate (dagger) of each gate tensor
    mpds_dag = [dag(t) for t in mpds]

    # 2) For each conjugated tensor, swap primed and unprimed indices:
    #    This ensures the “input” and “output” legs match the original gate
    for t in mpds_dag
        swapprime!(t, 0 => 1)
    end

    # 3) Reverse the order so the last gate of U becomes the first of dag(U)
    return reverse(mpds_dag)
end





"""
    deep!(psi_non_truncated::MPS, D::Int) → Vector{Vector{ITensor}}

Constructs a layered (depth-D) disentangling circuit for a given MPS by iteratively:
1. Truncating the current MPS to bond dimension ≤ 2,
2. Extracting its MPD gates via `generate_mpds`,
3. Conjugating & reversing them with `dagger!`, and
4. Applying them to advance to the next MPS.

# Arguments
- `psi_non_truncated::MPS`  
  Initial MPS whose entanglement we wish to peel away.
- `D::Int`  
  Desired circuit depth (number of layers).

# Returns
- `circuit_layers::Vector{Vector{ITensor}}`  
  A length-D vector, where each entry is a list of two-site (or boundary) ITensor gates 
  forming one disentangling layer.
"""
function deep!(psi_non_truncated, D)
    # Number of sites (should match length of psi_non_truncated)
    N = length(sites)  # ← ensure `sites` is in scope or replace with length(psi_non_truncated)

    # Preallocate storage for each circuit layer
    circuit_layers = Vector{Vector{ITensor}}(undef, D)
    
    # Start from the full (untruncated) MPS
    psi_current = copy(psi_non_truncated)

    # Build layer by layer
    for d in 1:D
        # 1) Lightly truncate to max bond dimension 2 to define a simplified circuit
        psi_trunc = truncate(psi_current, maxdim=2)

        # 2) Extract MPD gates from this truncated state
        mpds = generate_mpds(psi_trunc)

        # 3) Order the gates by bond index so we apply them left→right
        ordered_keys = sort(collect(keys(mpds)))
        mpds_vec = [mpds[n] for n in ordered_keys]

        # 4) Compute dagger(U) & reverse to form the inverse layer
        mpds_vec = dagger!(mpds_vec)

        # Store this layer
        circuit_layers[d] = mpds_vec

        # 5) Apply the disentangling layer to advance the MPS
        psi_current = apply(mpds_vec, psi_current)
    end
    
    return circuit_layers
end







"""
    reverse_deep!(circuit_layers::Vector{Vector{ITensor}}) → Vector{Vector{ITensor}}

Given a disentangling circuit `circuit_layers` (mapping `psi_non_truncated` to a product state), 
this function constructs the inverse entangling circuit that maps the product state `|0⟩` back 
to `psi_non_truncated`. It does so by:
1. Reversing the order of layers,
2. Reversing the order of gates within each layer, and
3. Taking the Hermitian conjugate + swapping primed indices of each gate.

# Arguments
- `circuit_layers::Vector{Vector{ITensor}}`  
  A length-D vector of layers, each a vector of two-site (or boundary) ITensors, representing the 
  disentangling circuit that maps `psi_non_truncated → |0⟩`.

# Returns
- `reversed_circuit::Vector{Vector{ITensor}}`  
  A length-D vector of layers that implements the inverse circuit: `|0⟩ → psi_non_truncated`.
"""
function reverse_deep!(circuit_layers::Vector{Vector{ITensor}})
    # Preallocate storage for the reversed circuit
    reversed_circuit = Vector{Vector{ITensor}}(undef, length(circuit_layers))
    
    # 1) Loop over layers in reverse order
    for (i, layer) in enumerate(reverse(circuit_layers))
        # Prepare a container for the reversed gates in this layer
        reversed_layer = Vector{ITensor}(undef, length(layer))
        
        # 2) Loop over gates in reverse order
        for (j, tensor) in enumerate(reverse(layer))
            # 3a) Take the Hermitian conjugate to invert the gate
            # 3b) Swap primed/unprimed indices so input/output legs match correctly
            reversed_layer[j] = swapprime(dag(tensor), 0 => 1)
        end
        
        # Store the reconstructed entangling layer
        reversed_circuit[i] = reversed_layer
    end
    
    return reversed_circuit
end






"""
    attract!(reversed_circuit::Vector{Vector{ITensor}}, product_state::MPS) → MPS

Applies an entangling circuit layer-by-layer to a simple product MPS (e.g., all spins down)
to reconstruct the target MPS `psi_non_truncated`. This “attraction” undoes the disentangling
procedure by sequentially contracting each layer of two-site gates.

# Arguments
- `reversed_circuit::Vector{Vector{ITensor}}`  
  A length-D vector of layers, each a vector of ITensor gates, representing the entangling circuit.
- `product_state::MPS`  
  The initial product-state MPS (e.g., |0⟩⊗…⊗|0⟩) to which the circuit is applied.

# Returns
- `state::MPS`  
  The MPS obtained after applying all layers, which should approximate `psi_non_truncated`.
"""
function attract!(reversed_circuit, product_state)
    # Number of layers in the circuit
    D = length(reversed_circuit)

    # Work on a copy so the original product_state is unchanged
    state = copy(product_state)
    
    # Sequentially apply each entangling layer
    for d in 1:D
        # 'apply' contracts the list of two-site gates with the current MPS
        state = apply(reversed_circuit[d], state)
    end

    # Return the reconstructed MPS
    return state
end




# ————————————————————————————————————————————————
# Example Usage: Verify Reconstruction Fidelity

# 1) Prepare the original MPS (ground state) and a simple product state
ψ_orig       = copy(psi_non_truncated)     # Ground-state MPS from DMRG
product_state = random_mps(sites, n -> "Dn")  # Or use a specific product state

# 2) Choose your disentangling circuit depth
D = 1  # Increase for a deeper circuit

# 3) Build the disentangling circuit layers
circuit_layers = deep!(ψ_orig, D)

# 4) Form the inverse (entangling) circuit
actual_circuit = reverse_deep!(circuit_layers)

# 5) Apply the entangling circuit to the product state
ψ_recon = attract!(actual_circuit, product_state)

# 6) Compute fidelity (|⟨ψ_orig | ψ_recon⟩|)
fidelity = abs(inner(ψ_orig, ψ_recon))
println("Reconstruction fidelity: ", fidelity)



